In [35]:
import os
import pandas as pd
import google.generativeai as genai
import json
import time
import re
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from dotenv import load_dotenv
from google.api_core import exceptions

# --- 1. SETUP & CONFIGURATION ---
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# ⚠️ MANUAL KEY OVERRIDE (Only if .env fails)
# GOOGLE_API_KEY = "AIzaSy....." 

if not GOOGLE_API_KEY:
    print("❌ Error: API Key not found. Please check .env or set manually.")
else:
    genai.configure(api_key=GOOGLE_API_KEY)
    print(f"✅ API Key Loaded. Configuring Gemini...")

# Use Flash model (Fastest/Cheapest)
model = genai.GenerativeModel('models/gemini-2.5-flash')


✅ API Key Loaded. Configuring Gemini...


In [36]:
# --- CELL 2: ROBUST API FUNCTIONS (The "Indefinite Wait" Fix) ---
import time
import random
import re
import json
from google.api_core import exceptions

def clean_and_parse_json(response_text):
    """Clean markdown fences and parse JSON."""
    try:
        text = re.sub(r"```json|```", "", response_text).strip()
        return json.loads(text)
    except json.JSONDecodeError:
        return None

def get_gemini_response_persistent(prompt, model):
    """
    Retries FOREVER until successful. 
    If Rate Limit is hit, it pauses and retries.
    """
    wait_time = 10  # Start with 10 seconds
    
    while True: # Loop forever until success
        try:
            return model.generate_content(prompt)
            
        except exceptions.ResourceExhausted:
            # We hit the limit. Wait, then retry.
            print(f"      ⏳ Limit hit. Waiting {wait_time}s...", end="\r")
            time.sleep(wait_time)
            
            # Increase wait time slightly (cap at 60s)
            wait_time = min(wait_time * 1.5, 60)
            
        except Exception as e:
            print(f"      ❌ Error: {e}")
            return None

def run_batch_strategy(df, prompt_func, batch_size=10, strategy_name="Strategy"):
    """
    Process reviews in batches with FORCED delays to stay safe.
    """
    print(f"\n🚀 Starting {strategy_name} (Batch Size: {batch_size})...")
    
    batches = [df[i:i + batch_size] for i in range(0, df.shape[0], batch_size)]
    all_predictions = []
    
    for i, batch in enumerate(batches):
        print(f"   📦 Batch {i+1}/{len(batches)}...", end=" ")
        
        reviews = batch['text'].tolist()
        prompt = prompt_func(reviews)
        
        # Call the new PERSISTENT function
        response = get_gemini_response_persistent(prompt, model)
        
        batch_preds = [3] * len(reviews)
        
        if response and response.text:
            data = clean_and_parse_json(response.text)
            if data and "ratings" in data and isinstance(data["ratings"], list):
                if len(data["ratings"]) == len(reviews):
                    batch_preds = data["ratings"]
                    print("✅ Success             ") 
                else:
                    print(f"⚠️ Count Mismatch")
            else:
                print("⚠️ Parse Error")
        else:
            print("❌ API Failed")
            
        all_predictions.extend(batch_preds)
        
        # CRITICAL: Force 10s sleep between batches to stay under the RPM limit
        time.sleep(10) 

    return all_predictions

In [37]:
# --- 3. DATA LOADING ---
try:
    df = pd.read_csv('yelp.csv')
    df = df[['text', 'stars']].dropna()
    sampled_df = df.sample(n=200, random_state=42).reset_index(drop=True)
except (FileNotFoundError, KeyError):
    print("⚠️ Dataset not found. Generating dummy data for testing.")
    data = {
        'text': [
            "The food was absolutely terrible and cold.", 
            "Amazing experience! Loved the ambiance.",
            "It was okay, nothing special.", 
            "Service was slow but burger was tasty.", 
            "Worst place I have ever been to."
        ] * 40, 
        'stars': [1, 5, 3, 4, 1] * 40
    }
    sampled_df = pd.DataFrame(data)

print(f"✅ Data Ready: {len(sampled_df)} reviews loaded.")

✅ Data Ready: 200 reviews loaded.


In [38]:
# --- 4. PROMPT DEFINITIONS ---

def prompt_batch_zero(reviews_list):
    formatted = "\n".join([f"Review {i+1}: {r}" for i, r in enumerate(reviews_list)])
    return f"""
    You are a sentiment analyzer. Classify these {len(reviews_list)} reviews (1-5 stars).
    Reviews:
    {formatted}
    Output STRICTLY a JSON object with a single list of integers:
    {{ "ratings": [star_1, star_2, star_3, ...] }}
    """

def prompt_batch_few(reviews_list):
    formatted = "\n".join([f"Review {i+1}: {r}" for i, r in enumerate(reviews_list)])
    return f"""
    Classify these Yelp reviews (1-5 stars) based on examples:
    Example 1: "Rude service, cold food." -> 1
    Example 2: "Okay meal, overpriced." -> 3
    Example 3: "Delicious! Great staff." -> 5
    
    Now classify these new reviews:
    {formatted}
    Output STRICTLY a JSON object with a single list of integers:
    {{ "ratings": [star_1, star_2, star_3, ...] }}
    """

def prompt_batch_cot(reviews_list):
    formatted = "\n".join([f"Review {i+1}: {r}" for i, r in enumerate(reviews_list)])
    return f"""
    Analyze the following reviews step-by-step.
    For each review, briefly weigh pros/cons and assign a score (1-5).
    
    Reviews:
    {formatted}
    
    Output STRICTLY a JSON object containing a list of integers:
    {{ 
      "reasoning": "Brief summary...",
      "ratings": [star_1, star_2, star_3, ...] 
    }}
    """

In [39]:
# --- CELL 5: RUN ALL STRATEGIES ---

# 1. Run Zero-Shot (Batch Size 10)
sampled_df['pred_zero_shot'] = run_batch_strategy(
    sampled_df, prompt_batch_zero, batch_size=10, strategy_name="Zero-Shot"
)

# 2. Run Few-Shot (Batch Size 10)
sampled_df['pred_few_shot'] = run_batch_strategy(
    sampled_df, prompt_batch_few, batch_size=10, strategy_name="Few-Shot"
)

# 3. Run Chain-of-Thought (Batch Size 5)
# We use smaller batches (5) because CoT generates more text
sampled_df['pred_cot'] = run_batch_strategy(
    sampled_df, prompt_batch_cot, batch_size=5, strategy_name="Chain-of-Thought"
)

print("\n🎉 ALL EXPERIMENTS COMPLETE!")


🚀 Starting Zero-Shot (Batch Size: 10)...


KeyboardInterrupt: 

In [ ]:
# --- 6. RESULTS & VISUALIZATION ---

# Calculate Accuracy
acc_zero = accuracy_score(sampled_df['stars'], sampled_df['pred_zero_shot'])
acc_few = accuracy_score(sampled_df['stars'], sampled_df['pred_few_shot'])
acc_cot = accuracy_score(sampled_df['stars'], sampled_df['pred_cot'])

# Create Summary DataFrame
results_df = pd.DataFrame({
    'Strategy': ['Zero-Shot', 'Few-Shot', 'Chain-of-Thought'],
    'Accuracy': [acc_zero, acc_few, acc_cot]
})

print("\n--- 🏆 FINAL RESULTS SUMMARY ---")
print(results_df)

# Plot Confusion Matrix (Chain of Thought)
plt.figure(figsize=(8, 6))
cm = confusion_matrix(sampled_df['stars'], sampled_df['pred_cot'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[1,2,3,4,5], yticklabels=[1,2,3,4,5])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix: Chain-of-Thought')
plt.show()

# Save File
sampled_df.to_csv("yelp_final_results.csv", index=False)
print("✅ Results saved to 'yelp_final_results.csv'")